In [ ]:
!pip install snowflake snowflake-ml-python snowflake-snowpark-python python-dotenv langchain-community

In [ ]:
!pip install crewai

In [1]:
!pip list | grep -i "snowflake"
!pip list | grep -i "langchain"
!pip list | grep -i "python-dotenv"

snowflake                                1.0.2
snowflake-connector-python               3.12.4
snowflake.core                           1.0.2
snowflake._legacy                        1.0.0
snowflake-ml-python                      1.7.2
snowflake-snowpark-python                1.26.0
langchain                                0.3.13
langchain-cohere                         0.3.4
langchain-community                      0.3.13
langchain-core                           0.3.28
langchain-experimental                   0.3.4
langchain-openai                         0.2.14
langchain-text-splitters                 0.3.3
python-dotenv                            1.0.1


In [11]:
!pip list | grep -i "llama-index"

llama-index                              0.12.8
llama-index-agent-openai                 0.4.1
llama-index-cli                          0.4.0
llama-index-core                         0.12.8
llama-index-embeddings-huggingface       0.4.0
llama-index-embeddings-openai            0.3.1
llama-index-indices-managed-llama-cloud  0.6.3
llama-index-llms-openai                  0.3.12
llama-index-multi-modal-llms-openai      0.4.1
llama-index-program-openai               0.3.1
llama-index-question-gen-openai          0.3.0
llama-index-readers-file                 0.4.1
llama-index-readers-github               0.5.0
llama-index-readers-llama-parse          0.4.0


In [8]:
!pip install langchain-mistralai llama-index-readers-github llama-index llama-index-embeddings-huggingface

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.1/242.1 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 61.0 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.20.3
    Uninstalling tokenizers-0.20.3:
      Successfully uninstalled tokenizers-0.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
chromadb 0.5.23 requires tokenizers<=0.20.3,>=0.13.2, but you have tokenizers 0.21.0 which is incompatible.


In [ ]:
!python -v

In [13]:
!python --version

Python 3.10.12


In [6]:
!pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 57.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 82.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.4 MB/s eta 0:00:00


In [16]:
import os
from typing import List
from crewai import Agent, Task, Crew
from langchain_community.document_loaders import PyPDFLoader
from snowflake.snowpark.session import Session
from snowflake.core import Root
import tempfile

/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_config.py:345: UserWarning: Valid config keys have changed in V2:
* 'fields' has been removed
  warnings.warn(message, UserWarning)


In [17]:
from google.colab import userdata  # for Google Colab authentication

connection_params = {
    "account": userdata.get('SNOWFLAKE_ACCOUNT'),
    "user": userdata.get('SNOWFLAKE_USER'),
    "password": userdata.get('SNOWFLAKE_USER_PASSWORD'),
    "role": userdata.get('SNOWFLAKE_ROLE'),
    "database": userdata.get('SNOWFLAKE_DATABASE'),
    "schema": userdata.get('SNOWFLAKE_SCHEMA'),
    "warehouse": userdata.get('SNOWFLAKE_WAREHOUSE')
}
snowpark_session = Session.builder.configs(connection_params).create()

In [ ]:
connection_params

In [8]:
import os
from typing import List
from crewai import Agent, Task, Crew
from langchain.tools import Tool
from langchain_community.document_loaders import PyPDFLoader
from snowflake.snowpark.session import Session
from snowflake.core import Root
import tempfile

class PDFProcessingAgent:
    def __init__(self, snowpark_session: Session):
        self.session = snowpark_session
        self.root = Root(self.session)

        # Initialize Snowflake table for storing PDF chunks
        self.init_snowflake_storage()

    def init_snowflake_storage(self):
        """Initialize Snowflake table for storing PDF text chunks"""
        # Set the database context
        self.session.sql("USE DATABASE BADSCIENTIST_DEMO").collect()
        self.session.sql("USE SCHEMA NOTEBOOKS").collect()

        # Create table for storing PDF chunks
        self.session.sql("""
            CREATE TABLE IF NOT EXISTS pdf_docs (
                id INTEGER AUTOINCREMENT,
                doc_text STRING,
                metadata VARIANT
            )
        """).collect()

        # Create or replace Cortex Search service
        self.session.sql("""
            CREATE OR REPLACE CORTEX SEARCH SERVICE BADSCIENTIST_DEMO.NOTEBOOKS.pdf_docs_svc
            ON doc_text
            WAREHOUSE = compute_wh
            TARGET_LAG = '1 hour'
            AS
                SELECT
                    doc_text
                FROM pdf_docs
        """).collect()

    def process_pdf(self, pdf_path: str) -> List[str]:
        """Process PDF file and split into chunks"""
        loader = PyPDFLoader(pdf_path)
        pages = loader.load()

        # Split pages into smaller chunks for better retrieval
        chunks = []
        for page in pages:
            # Split into paragraphs or smaller segments
            text_chunks = self._split_into_chunks(page.page_content)
            chunks.extend(text_chunks)

        return chunks

    def _split_into_chunks(self, text: str, chunk_size: int = 1000) -> List[str]:
        """Split text into smaller chunks"""
        words = text.split()
        chunks = []
        current_chunk = []
        current_size = 0

        for word in words:
            if current_size + len(word) > chunk_size:
                chunks.append(" ".join(current_chunk))
                current_chunk = [word]
                current_size = len(word)
            else:
                current_chunk.append(word)
                current_size += len(word)

        if current_chunk:
            chunks.append(" ".join(current_chunk))

        return chunks

    def store_chunks(self, chunks: List[str]):
        """Store text chunks in Snowflake"""
        for chunk in chunks:
            # Insert chunk into Snowflake
            self.session.sql("""
                INSERT INTO pdf_docs (doc_text, metadata)
                SELECT column1, PARSE_JSON(column2)
                FROM VALUES(?, ?)
            """, params=(chunk, '{}')).collect()

class PDFQueryAgent:
    def __init__(self, snowpark_session: Session):
        self.session = snowpark_session
        self.root = Root(self.session)

    def query(self, question: str, num_chunks: int = 4) -> dict:
        """Query the stored PDF content using Cortex Search"""
        # Get relevant chunks using Cortex Search
        search_service = (
            self.root
            .databases[self.session.get_current_database()]
            .schemas[self.session.get_current_schema()]
            .cortex_search_services['pdf_docs_svc']
        )

        results = search_service.search(
            query=question,
            columns=["doc_text"],
            limit=num_chunks
        )

        context = "\n\n".join([r["doc_text"] for r in results.results])

        # Generate response using Snowflake's LLM
        prompt = f"""
        Based on the following context, answer the question.

        Context:
        {context}

        Question: {question}

        Answer:
        """
        print(prompt)
        response = self.session.sql(
            "SELECT snowflake.cortex.complete(?, ?)",
            params=("mistral-large", prompt)
        ).collect()[0][0]

        return {
            'question': question,
            'context': context,
            'response': response
        }

def create_pdf_tools(snowpark_session: Session):
    """Create LangChain tools for PDF processing and querying"""
    pdf_processor = PDFProcessingAgent(snowpark_session)
    pdf_querier = PDFQueryAgent(snowpark_session)

    process_tool = Tool(
        name="process_pdf",
        func=lambda pdf_path: pdf_processor.process_pdf(pdf_path),
        description="Process a PDF file and split it into chunks"
    )

    store_tool = Tool(
        name="store_chunks",
        func=lambda chunks: pdf_processor.store_chunks(chunks),
        description="Store text chunks in Snowflake database"
    )

    query_tool = Tool(
        name="query_pdf",
        func=lambda question: pdf_querier.query(question),
        description="Query PDF content and get answers"
    )

    return [process_tool, store_tool, query_tool]

# Create CREW AI agents
def create_pdf_crew(snowpark_session: Session):
    tools = create_pdf_tools(snowpark_session)

    # PDF Processing Agent
    pdf_processor = Agent(
        name="PDF Processor",
        role="Processes and stores PDF documents in Snowflake",
        goal="Efficiently process PDFs and store them for later retrieval",
        backstory="Expert at handling PDFs and managing document storage",
        tools=tools[:2]  # process_pdf and store_chunks tools
    )

    # Query Agent
    query_agent = Agent(
        name="Query Expert",
        role="Answers questions about stored PDF content",
        goal="Provide accurate answers based on stored PDF content",
        backstory="Specialist in information retrieval and question answering",
        tools=[tools[2]]  # query_pdf tool
    )

    # Create crew
    crew = Crew(
        agents=[pdf_processor, query_agent],
        tasks=[
            Task(
                description="Process and store PDF documents",
                agent=pdf_processor,
                expected_output="A confirmation message that the PDF has been processed and stored in the database"
            ),
            Task(
                description="Answer questions about PDF content",
                agent=query_agent,
                expected_output="A detailed answer to the user's question based on the content stored in the database"
            )
        ]
    )

    return crew

In [5]:
def handle_pdf_upload(pdf_file):
    try:
        with tempfile.NamedTemporaryFile(delete=False, suffix='.pdf') as tmp_file:
            tmp_file.write(pdf_file.read())
            tmp_path = tmp_file.name

        processor = PDFProcessingAgent(snowpark_session)
        chunks = processor.process_pdf(tmp_path)
        processor.store_chunks(chunks)
        os.unlink(tmp_path)

        print(f"Successfully processed PDF and stored {len(chunks)} chunks")
        return "PDF processed and stored successfully!"
    except Exception as e:
        print(f"Error processing PDF: {str(e)}")
        raise

# Query the content
def ask_question(question: str):
    query_agent = PDFQueryAgent(snowpark_session)
    result = query_agent.query(question)
    return result['response']

In [9]:
# Initialize
snowpark_session = Session.builder.configs(connection_params).create()
crew = create_pdf_crew(snowpark_session)

# Process a PDF
with open('/content/RAG-paper.pdf', 'rb') as f:
    result = handle_pdf_upload(f)
print(result)

Successfully processed PDF and stored 68 chunks
PDF processed and stored successfully!


In [12]:
# Ask questions
response = ask_question("What is retrieval, how can I use python to code this concept")
print(response)


        Based on the following context, answer the question.
        
        Context:
        performance as more documents are retrieved. Center: Retrieval recall perfor- mance in NQ. Right: MS-MARCO Bleu-1 and Rouge-L as more documents are retrieved. 5 Related Work Single-Task Retrieval Prior work has shown that retrieval improves performance across a variety of NLP tasks when considered in isolation. Such tasks include open-domain question answering [5, 29], fact checking [ 56], fact completion [ 48], long-form question answering [ 12], Wikipedia article generation [36], dialogue [ 41, 65, 9, 13], translation [ 17], and language modeling [ 19, 27]. Our work uniﬁes previous successes in incorporating retrieval into individual tasks, showing that a single retrieval-based architecture is capable of achieving strong performance across several tasks. 8

z∈top-k(p(·|x)) pη(z|x)pθ(y|x,z) = ∑ z∈top-k(p(·|x)) pη(z|x) N∏ i pθ(yi|x,z,y 1:i−1) RAG-Token Model In the RAG-Token model we can dra